## 5.2 Auxiliary particle filter

앞장에서 소개한 입자필터는 일반적인 상태공간 모델에 적용됩니다. 하지만 이 입자필터의 성능은 중요도 전이 밀도에 따라 급격히 달라집니다. DLM의 경우 최적 중요도 커널을 구할 수 있고, 이를 사용하면 필터링 분포에 대한 꽤 정확한 추정을 할 수 있지만, 일반적인 상태공간 모델의 경우는 그렇지 않으며, 유효 중요도 전이 밀도를 수정하는 것은 훨씬 어려운 문제입니다. Pitt과 Shepard (1999)는 이러한 어려운 점을 극복하기 위하여 보조 입자 필터 알고리즘을 제안했습니다. 완전하게 정의된 DLM의 경우에는 별로 필요하지 않지만, Liu와 West(2001)는 보조 입자 필터 알고리즘을 확장하여, 모델에 알지 못하는 변수가 포함된 DLM의 경우에도 매우 유용하게 사용될 수 있음을 보였습니다. 본 장에서는 Pitt 과 Shephard의 보조 입자 필터에 대해 소개하고 다음 장에서는 알지 못하는 모델 변수를 다루기 위해 확장시킨 Liu와 West의 보조 입자 필터에 대해 설명하겠습니다.

시간 $t-1$에서 결합 스무딩 분포 $\pi(\theta_{0:t-1}|y_{1:t-1})$에 대한 이산 추정치 $\hat{\pi}_{t-1}=\sum_{i=1}^{N} w_{t-1}^{(i)}\delta_{\theta_{0:t-1}^{(i)}}$를 알 수 있다고 가정해 봅시다. 새로운 관측치 정보를 얻었을 때 이 추정 스무딩 분포를 업데이트 하는 것이 목적입니다. 다시말해 시간 $t$에서 결합 스무딩 분포 $\pi(\theta_{0:t}|y_{1:t})$에 대한 이산 추정치 $\hat{\pi}_t$를 얻는 것이 목적입니다. 시간 $t$에서 결합 스무딩 분포 $\pi(\theta_{0:t}|y_{1:t})$는 다음과 같이 표현할 수 있습니다.

$$\pi(\theta_{0:t}|y_{1:t})\propto\pi(\theta_{0:t},y_t|y_{1:t-1})$$

$$=\pi(y_t|\theta_{0:t},y_{1:t-1})\cdot\pi(\theta_t|\theta_{0:t},y_{1:t-1})\cdot\pi(\theta_{0:t-1}|y_{1:t-1})$$

$$=\pi(y_t|\theta_t)\cdot\pi(\theta_t|\theta_{t-1})\cdot(\pi(\theta_{0:t-1}|y_{1:t-1})$$

$$\approx\pi(y_t|\theta_t)\cdot\pi(\theta_t|\theta_{t-1})\cdot\hat{\pi}_{t-1}(\theta_{0:t-1})$$

$$=\sum_{i=1}^{N}w_{t-1}^{(i)}\pi(y_t|\theta_t)\pi(\theta_t|\theta_{t-1}^{(i)})\delta_{\theta_{0:t-1}^{(i)}}$$

상기 마지막 항은 처음 $t$의 성분은 이산이고 마지막 $\theta_t$의 성분은 연속인 $\theta_{0:t}$에 대한 정규화 되지 않은 분포를 나타냅니다. 이 분포는 $\pi(\theta_{0:t}|y_{1:t})$에 대한 추정치 이며, 중요도 샘플링 단계에서 목표 분포로 사용할 수 있습니다. 이 목표 분포 혼합 분포일 수 있으로, 합을 제거하기 위한 표준 방법으로 아래와 같이 $\{1,\dots,N\}$값 중 한개로 잠재변수 $I$를 적용 합니다.

$$P(I=i)=w_{t-1}^{(i)}$$

$$\theta_{0:t}|I=i\sim C\pi(y_t|\theta_t)\pi(\theta_t|\theta_{t-1}^{(i)})\delta_{\theta_{0:t-1}^{(i)}}$$

따라서 확장된 목표 분포는 다음과 같이 됩니다.

$$\pi^{aux}(\theta_{0:t},i|y_{1:t})\propto w_{t-1}^{(i)}\pi(y_t|\theta_t)\pi(\theta_t|\theta_{t-1}^{(i)})\delta_{\theta_{0:t-1}^{(i)}}$$

이 목표 분포에 대하여 Pitt과 Shephard가 제안한 중요도 밀도는 다음과 같습니다.

$$g_t(\theta_{0:t},i|y_{1:t})\propto w_{t-1}^{(i)} \pi(y_t|\hat{\theta}_t^{(i)})\pi(\theta_t|\theta_{t-1}^{(i)})\delta_{\theta_{0:t-1}^{(i)}}$$

여기서 $\hat{\theta}_t^{(i)}$는 $\pi(\theta_t|\theta_{t-1}=\theta_{t-1}^{(i)})$의 중심 값 (평균 또는 모드)입니다. 다음의 과정을 $k=1,\dots,N$까지 반복하면 $g_t$로 부터 샘플을 얻을 수 있습니다. 

1. 다음을 사용해 분류 변수 $I_k$를 샘플링 한다.

$$P(I_k=i)\propto w_{t-1}^{(i)} \pi(y_t|\hat{\theta}_t^{(i)}),\;\;\;\;i=1,\dots,N$$

2. $I_k=i$가 주어지면 다음을 샘플링 한다.

$$\theta_t^{(k)} \sim \pi(\theta_t|\theta_{t-1}^{(i)})$$

그리고 $\theta_{0:t}^{(k)}=(\theta_{0:t-1}^{(i)},\theta_t^{(k)})$로 설정한다.

$g_t$에서 $k$번째로 샘플링한 중요도 가중치는 다음에 비례합니다.

$$\tilde{w}_t^{(k)}=\frac{w_{t-}^{(I_k)}\pi(y_t|\theta_t^{(k)})\pi(\theta_t^{(k)}|\theta_{t-1}^{(k)})}{w_{t-}^{(I_k)}\pi(y_t|\hat{\theta}_t^{(k)})\pi(\theta_t^{(k)}|\theta_{t-1}^{(k)})}=\frac{\pi(y_t|\theta_t^{(k)})}{\pi(y_t|\hat{\theta}_t^{(k)})}$$

$\tilde{w}_t^{(k)}$를 정규화 하고, 분류 변수 $I_k$들을 제거하면, 최종적으로 시간 $t$에서의 결합 스무딩 분포에 대한 이산 추정치를 얻게 됩니다.

$$\hat{\pi}_t(\theta_{0:t})=\sum_{i=1}^N w_t^{(i)}\delta_{\theta_{0:t}^{(i)}}\approx \pi(\theta_{0:t}|y_{1:t})$$

5.1장의 표준 알고리즘과 마찬가지로, 리샘플 단계는 유효 샘플 크기가 특정 기준보다 작을 때 수행합니다. 알고리즘 5.2는 보조 입자 필터를 요약한 것입니다.

## 알고리즘 5.2 보조 입자 필터 알고리즘에 대한 요약

--------------------------------------------------------------------------------------------------------
<div><h4>0. 초기화 : $\pi(\theta_0)$에서 독립적으로 $N$개의 변수 $\theta_0^{(1)},\dots,\theta_0^{(N)}$를 샘플링 하고 다음과 같이 가중치를 설정한다.</h4></div>
<br>
<div>
$$w_0^{(i)}=N^{-1},\;\;\;\;i=1,\dots,N$$
</div>
<br>
<div><h4>1. For $t=1,\dots,T$</h4></div>

<div><h5>&nbsp;&nbsp;1.1) For $i=1,\dots,N$</h5></div>


<div><h6>&nbsp;&nbsp;&nbsp;&nbsp;* $P(I_k=i)\propto w_{t-1}^{(i)}\pi(y_t|\hat{\theta_t^{(i)}})$에서 $I_k$를 샘플링 한다.</h6></div>

<div><h6>&nbsp;&nbsp;&nbsp;&nbsp;* $\pi(\theta_t|\theta_{t-1}=\theta_{t-1}^{(I_k)}$에서 $\theta_t^{(k)}$를 샘플링 하고 다음과 같이 설정한다.</h6></div>

<br>
<div>$$\theta_{0:t}^{(k)}=(\theta_{0:t-1}^{(I_k)},\theta_t^{(k)})$$</div>
<br>    

<br>
<div>$$\tilde{w}_t^{(k)}=\frac{\pi(y_t|\theta_t^{(k)})}{\pi(y_t|\hat{\theta}_t^{(k)})}$$</div>
<br>    
    
<div><h5>&nbsp;&nbsp;1.2) 다음과 같이 가중치를 정규화 한다.</h5></div>
  
<br>
<div>$$w_t^{(i)}=\frac{\tilde{w}_t^{(i)}}{\sum_{j=1}^N \tilde{w}_t^{(j)}}$$</div>
<br>  
  
<div><h5>&nbsp;&nbsp;1.3) 다음과 같이 유효샘플크기를 계산한다.</h5></div>
  
<br>  
<div>$$N_{eff}=\left(\sum_{i=1}^N (w_t^{(i)})^2\right)^{-1}$$</div>
<br>  
  
<div><h5>&nbsp;&nbsp;1.4) $N_{eff} < N_0$이면 다음과 같이 리샘플을 수행한다.</h5></div>
  
  
<div><h6>&nbsp;&nbsp;&nbsp;&nbsp;* 이산 분포로 부터 $N$개의 샘플을 추출한다.</h6></div>
    
<br>    
<div>$$P(\theta_{0:t}=\theta_{0:t}^{(i)})=w_t^{(i)},\;\;\;\;i=1,\dots,N$$</div>
<br>    
    
<div><h6>&nbsp;&nbsp;&nbsp;&nbsp;그리고 이 샘플들을 다시 라벨링 한다.</h6></div>
    
<br>    
<div>$$\theta_{0:t}^{(1)},\dots,\theta_{0:t}^{(N)}$$</div>
<br>    
    
<div><h6>&nbsp;&nbsp;&nbsp;&nbsp;* 가중치를 초기화 한다 : $w_t^{(i)}=N^{-1},\;\;i=1,\dots,N$</h6></div>
    
    
<div><h5>&nbsp;&nbsp;1.5) $\hat{\pi}_t=\sum_{i=1}^N w_t^{(i)}\delta_{\theta_{0:t}^{(i)}}$로 설정한다.</h5></div>

----------------------------------------------------------------------------------------------------

앞장에서 설명한 단순 직접 알고리즘 대비 보조 입자 필터의 주된 장점은 효율성을 적게 상실하면서도 $\theta_t$를 샘플링 하는데 한단계 이전 분포인 $\pi(\theta_t|\theta_{t-1})$를 사용할 수 있다는 점입니다. $g_t$로 부터 샘플링을 할 때 첫번째 단계의 역할은 새로운 관측치 $y_t$를 고려할 때 $\theta_t$로 될 가능성이 높은 $\theta_{t-1}$을 미리 선택하는 것이라고 할 수 있습니다. 이러한 방법을 사용하면 사전 분포 $\pi(\theta_t|\theta_{t-1})$와 가능도함수 $\pi(y_t|\theta_t)$의 충돌을 최소화 할 수 있습니다. 일반적인 상태공간 모델의 경우 사전 분포는 대부분 얻을 수 있지만, 최적의 측정 커널에서 샘플을 하는 것은 DLM의 경우와 달리 불가능한 경우가 많습니다. 따라서 이를 보조 입자 필터 알고리즘을 통해 해결하면 효율적이고 단순한 방법이 될 수 있습니다.